### Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

### Rusty Bargain is interested in:

### - the quality of the prediction;
### - the speed of the prediction;
### - the time required for training

## Data preparation

In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('/Users/joemeyer/Library/Mobile Documents/com~apple~CloudDocs/Work Folder/TripleTen_Sprint12/data/car_data.csv')

In [3]:
df.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
5,04/04/2016 17:36,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,04/04/2016 00:00,0,33775,06/04/2016 19:17
6,01/04/2016 20:48,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,01/04/2016 00:00,0,67112,05/04/2016 18:18
7,21/03/2016 18:54,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,21/03/2016 00:00,0,19348,25/03/2016 16:47
8,04/04/2016 23:42,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,04/04/2016 00:00,0,94505,04/04/2016 23:42
9,17/03/2016 10:53,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,17/03/2016 00:00,0,27472,31/03/2016 17:17


In [4]:
df = df.rename(columns={'DateCrawled': 'date_crawled', 'VehicleType': 'vehicle_type', 'Price': 'price', 'RegistrationYear': 'registration_year', 'Gearbox': 'gearbox',
                   'Power': 'power', 'Model': 'model', 'Mileage': 'mileage', 'RegistrationMonth': 'registration_month', 'FuelType': 'fuel_type',
                   'Brand': 'brand', 'NotRepaired': 'not_repaired', 'DateCreated': 'date_created', 'NumberOfPictures': 'number_of_pictures',
                   'PostalCode': 'postal_code', 'LastSeen': 'last_seen'})

In [ ]:
# Running info() again to confirm that changes to column names went through
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [6]:
df.describe()

,price,registration_year,power,mileage,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Here I can gauge what needs to be dropped and how to better clean up the data. For example the earliest registration year being 1000, number of photos is all 0, and the minimum price amount being 0 are some good examples

In [7]:
df.duplicated().sum()

np.int64(262)

In [8]:
# Since the duplicated amount is so small compared to the overall data I think it's safe enough to drop these rows
df.drop_duplicates(inplace=True)

In [9]:
df.isna().sum()

date_crawled              0
price                     0
vehicle_type          37484
registration_year         0
gearbox               19830
power                     0
model                 19701
mileage                   0
registration_month        0
fuel_type             32889
brand                     0
not_repaired          71145
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64

In [10]:
# Zero pictures all around does not need to be included
df = df.drop('number_of_pictures', axis=1)

# Drop Registration month that equals zero since this is implausible
df = df[(df['registration_month'] != 0)]

# Minimum price being 0 will mess with model training
df = df[df['price'] > 0]

# A lot of the power values were implausible so I made the range narrower
df = df[(df['power'] >= 10) & (df['power'] <= 1000)]

# Registration year values going all the way back to 1000 and going all the way into the future 9999
df = df[(df['registration_year'] >= 1950) & (df['registration_year'] <= 2025)]

# Fill missing categorical column values with unknown
categorical_cols = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'not_repaired']
df[categorical_cols] = df[categorical_cols].fillna('unknown')

In [ ]:
# Running info() again to confirm that the non-null count values match 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286385 entries, 1 to 354368
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        286385 non-null  object
 1   price               286385 non-null  int64 
 2   vehicle_type        286385 non-null  object
 3   registration_year   286385 non-null  int64 
 4   gearbox             286385 non-null  object
 5   power               286385 non-null  int64 
 6   model               286385 non-null  object
 7   mileage             286385 non-null  int64 
 8   registration_month  286385 non-null  int64 
 9   fuel_type           286385 non-null  object
 10  brand               286385 non-null  object
 11  not_repaired        286385 non-null  object
 12  date_created        286385 non-null  object
 13  postal_code         286385 non-null  int64 
 14  last_seen           286385 non-null  object
dtypes: int64(6), object(9)
memory usage: 35.0+ MB


In [ ]:
# These columns won't contribute to model testing, so we will drop
df = df.drop(columns=['date_crawled', 'date_created', 'postal_code', 'last_seen'], axis=1)

In [ ]:
# Last show of the columns that are left with NaN values filled with 'unknown'

df.head(10)

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no
8,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,unknown
10,2000,sedan,2004,manual,105,3_reihe,150000,12,petrol,mazda,no
11,2799,wagon,2005,manual,140,passat,150000,12,gasoline,volkswagen,yes
12,999,wagon,1995,manual,115,passat,150000,11,petrol,volkswagen,unknown


### Data Preparation Conclusions

Steps Taken:
- Loaded dataset and analyzed column type and non-null values
- Renamed each column to lowercase and snake_case where applicaable 
- Dropped implausible values from data and duplicate rows, as well as filled in missing values with 'unknown' as I couldn't think of a better option
- Lastly dropped columns that would not be relevant in the model training and development. Meaning, the date that the user's profile was active or made, will not affect the price of the car.

## Model training

In [ ]:
# Features
x = df.drop('price', axis=1)

# Target
y = df['price']

x_train, x_valid, y_train, y_valid = train_test_split(
    x, y, test_size=0.25, random_state=12345)

In [15]:
x_train = pd.get_dummies(x_train, drop_first=True)
x_test = pd.get_dummies(x_valid, drop_first=True)

# Aligning columns 
x_train, x_valid = x_train.align(x_valid, join="left", axis=1, fill_value=0)

In [16]:
model_lr = LinearRegression()

start_time = time.time()

model_lr.fit(x_train, y_train)

train_time = time.time() - start_time

print(f'Training time took: {train_time:.2f} seconds')

Training time took: 1.59 seconds


In [17]:
start_time = time.time()

xlr_pred = model_lr.predict(x_valid)

predict_time = time.time() - start_time

print(f'Prediction time took: {predict_time:.2f} seconds')

Prediction time took: 0.07 seconds


In [18]:
rmse_lr = root_mean_squared_error(y_valid, xlr_pred)

print(f'RMSE for Linear Regression is: {rmse_lr:.2f}')

RMSE for Linear Regression is: 3331.90


This is the baseline for our other models. Linear regression(LR) is off on price by about $3,331.90. Random Forest Regression(RF), which any advanced model will most likely do better than LR, will hopefully get a lower RMSE. If it doesn't get a lower RMSE than LR, then it tells us something is wrong with the preprocessing or possible overfitting of the model 

In [19]:
model_rf = RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1, min_samples_leaf=3, random_state=12345)

start_time = time.time()

model_rf.fit(x_train, y_train)

train_time = time.time() - start_time

print(f'Training time took: {train_time:.2f} seconds')

Training time took: 28.53 seconds


In [20]:
start_time = time.time()

xrf_pred = model_rf.predict(x_valid)

predict_time = time.time() - start_time

print(f'Prediction time took: {predict_time:.2f} seconds')

Prediction time took: 0.10 seconds


In [21]:
rmse_rf = root_mean_squared_error(y_valid, xrf_pred)

print(f'RMSE for Random Forest Regression is: {rmse_rf:.2f}')

RMSE for Random Forest Regression is: 2294.27


The RMSE went down by about $1,000. This is a huge improvement compared to our baseline of 3331.90. Obviously the reason why RF is performing better is because it handles nonlinear data better than LR, it also aggregates many trees and reduces the variance. It also is more robust to overfitting and would perform better than a single tree like Decision Tree Regression. Next is to use gradient boosting which is a more powerful and faster and should improve the quality and speed of the predictions

In [22]:
lgbm = lgb.LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=7, num_leaves=50, n_jobs=-1, random_state=12345)

start_time = time.time()

lgbm.fit(x_train, y_train)

train_time = time.time() - start_time

print(f'Training time: {train_time:.2f} seconds')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 927
[LightGBM] [Info] Number of data points in the train set: 214788, number of used features: 293
[LightGBM] [Info] Start training from score 4977.643975
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [ ]:
start_time = time.time()

xlgbm_pred = lgbm.predict(x_valid)

predict_time = time.time() - start_time

print(f'Prediction time is: {predict_time:.2f} seconds')

Prediction time is: 0.28 seconds


In [24]:
rmse_lgbm = root_mean_squared_error(y_valid, xlgbm_pred)
print(f'RMSE for LightGBM is: {rmse_lgbm:.2f}')

RMSE for LightGBM is: 2307.97


LightGBM is basically on par with RF, RMSE being only ~$10 above what RF got. My next step is to try a Catboost model to see if we can get a better RMSE score and to compare with the other models 

In [25]:
model_cat = CatBoostRegressor(random_state=12345, iterations=500, learning_rate=0.05, depth=6, verbose=0)
start_time = time.time()

model_cat.fit(x_train, y_train)
train_time = time.time() - start_time

print(f'Training time took: {train_time:.2f} seconds')

Training time took: 2.80 seconds


In [26]:
start_time = time.time()
cat_pred = model_cat.predict(x_valid)

predict_time = time.time() - start_time

print(f'Prediction time took: {predict_time:.2f} seconds')

Prediction time took: 0.02 seconds


In [27]:
rmse_cat = root_mean_squared_error(y_valid, cat_pred)
print(f'RMSE for CatBoost is: {rmse_cat:.2f}')

RMSE for CatBoost is: 2376.05


### Model Training Conclusions

We tested a total of 4 models
- Linear Regression (RMSE: 3331.90)
- Random Forest (RMSE: 2294.27)
- LightGBM (RMSE: 2307.97)
- CatBoost (RMSE: 2376.05)

Random Fprest had the best overall result with LightGBM being a very close second. Linear Regression acted as our baseline and sanity check for the other models. The models improving upon Linear Regression showed that the preprocessing and encoding was working as intended. 

## Model analysis

#### Models Tested
- Linear Regression
- Random Forest Regression
- LightGBM
- CatBoost Regression


#### Evauluation Metric
We used RMSE as our evaluation metric as it measures the average prediction error, and in our case, average prediction error in euros. As such, lower RMSE 
means the more accurate the model was.


#### Model Performance Summary
| Model | RMSE |
| --- | --- |
| Linear Regression | 3331.90 |
| Tuned Random Forest | 2294.27 |
|  Tuned LightGBM | 2307.97 |
| CatBoost | 2376.05 |

##### Key Findings
- Linear Regression has the worst RMSE, which is to be expected since it doesn't handle non-linear relationships very well
- Random Forest with tuning had the best RMSE
- LightGBM with tuning was a very close second to Random Forest with an almost identical performance

#### Training and Prediction Speed (in seconds)
| Model | Training Speed | Prediction Speed |
| ---| --- | --- |
| Linear Regression | 1.59 | 0.07 |
| Random Forest | 28.53 | 0.10 | 
| LightGBM | 2.41 | 0.28 | 
| CatBoost | 2.8 | 0.02 |

##### Interpretation
- While ensemble methods like Random Forest and gradient boosting models significantly improved prediction accuracy over Linear Regression, they required longer training times. Among the boosting models, LightGBM offered a good balance between training time and accuracy, while CatBoost provided very fast prediction times. In production scenarios where prediction latency is critical, CatBoost could be preferable despite slightly higher RMSE.

### Conclusion

Based on the balance of prediction quality, training speed, and prediction speed, the most effective models were:

- Random Forest Regressor 
- LightGBM Regressor

These models achieved RMSE around $2,300, representing a ~30% improvement in error compared to the baseline Linear Regression.

# Checklist

Type 'x' to check. Then press Shift+Enter.

- [x]  Jupyter Notebook is open
- [x]  Code is error free
- [x]  The cells with the code have been arranged in order of execution
- [x]  The data has been downloaded and prepared
- [x]  The models have been trained
- [x]  The analysis of speed and quality of the models has been performed